In [ ]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124
!pip install transformers accelerate huggingface_hub pandas optuna pyproj einops geopy matplotlib kaggle  geopandas cartopy

Looking in indexes: https://download.pytorch.org/whl/cu124


In [15]:
!nvidia-smi

Sun May 25 08:02:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from huggingface_hub import snapshot_download, hf_hub_download, login

login(token="")

path = snapshot_download(
    repo_id="tduongvn/ACMMM25-Geolocation",
    repo_type="dataset",
    allow_patterns=["*.tar"],
    local_dir="data/mp16/",
    use_auth_token=True  # will use your local token from `huggingface-cli login`
)
print(f"Downloaded dataset to {path}")

files = [
    "metadata/MP16_Pro_filtered.csv",
    "metadata/MP16_Pro_places365.csv",
    "metadata/mp16_urls.csv"
]
for file in files:
    path = hf_hub_download(
        repo_id="Jia-py/MP16-Pro",
        filename=file,
        repo_type="dataset",
        local_dir="data/mp16/",
        use_auth_token=True  # will use your local token from `huggingface-cli login`
    )
    print(f"Downloaded {file} to {path}")


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

filtered_mp16.tar:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

Downloaded dataset to /kaggle/working/G3-Original/data/mp16


MP16_Pro_filtered.csv:   0%|          | 0.00/747M [00:00<?, ?B/s]

Downloaded metadata/MP16_Pro_filtered.csv to data/mp16/metadata/MP16_Pro_filtered.csv


MP16_Pro_places365.csv:   0%|          | 0.00/859M [00:00<?, ?B/s]

Downloaded metadata/MP16_Pro_places365.csv to data/mp16/metadata/MP16_Pro_places365.csv


mp16_urls.csv:   0%|          | 0.00/385M [00:00<?, ?B/s]

Downloaded metadata/mp16_urls.csv to data/mp16/metadata/mp16_urls.csv


In [17]:
!mv data/mp16/metadata/*.csv data/mp16

In [18]:
!ls data/mp16/

filtered_mp16.tar  MP16_Pro_filtered.csv   mp16_urls.csv
metadata	   MP16_Pro_places365.csv


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

api.dataset_download_files('lctngdng/im2gps3k', path='data/im2gps3k', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/lctngdng/im2gps3k


In [20]:
import geopandas as gpd
from shapely.geometry import Point
import cartopy.io.shapereader as shpreader

# Load Natural Earth country polygons
shp_path = shpreader.natural_earth(
    resolution='110m',
    category='cultural',
    name='admin_0_countries'
)
world = gpd.read_file(shp_path)[["ADMIN", "geometry"]].to_crs(epsg=4326)
# Build spatial index for performance
sindex = world.sindex

# Function returns country name for a given latitude/longitude
def point_to_country(lat, lon, countries=world, index=sindex):
    pt = Point(lon, lat)
    # Find candidate polygons via spatial index
    candidates = list(index.intersection(pt.bounds))
    # Check strict land contains
    for idx in candidates:
        if countries.geometry.iloc[idx].contains(pt):
            return countries.ADMIN.iloc[idx]
    # Fallback to intersects for border/water cases
    for idx in candidates:
        if countries.geometry.iloc[idx].intersects(pt):
            return countries.ADMIN.iloc[idx]
    # No match found
    return None

In [21]:
import pandas as pd
df = pd.read_csv("data/im2gps3k/im2gps3k_places365.csv")
df.head()

,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban
0,1000269685_e60e9cdfb4_1125_78841376@N00.jpg,78841376@N00,32.325436,-64.764404,2,12,353,0.274242,0.045113,0.680645
1,1000304467_1a75a200b1_1296_78841376@N00.jpg,78841376@N00,32.325436,-64.764404,0,4,325,0.414407,0.220912,0.364681
2,1001048550_8e4b47d165_1051_78841376@N00.jpg,78841376@N00,32.325436,-64.764404,1,8,36,0.007326,0.969903,0.022771
3,1005977048_5ccf8b05d3_1201_91728102@N00.jpg,91728102@N00,29.976052,122.390356,2,6,273,0.003868,0.476665,0.519468
4,1008804117_ce4e6fef8a_1349_97522422@N00.jpg,97522422@N00,46.478536,30.758714,0,0,198,0.973878,0.002453,0.023670


In [22]:
def is_target_country(row):
    lat, lon = row["LAT"], row["LON"]
    return point_to_country(lat, lon) in ["Ukraine", "Israel", "Russia", "Palestine"]

# apply row‐wise and get a boolean mask
mask = df.apply(is_target_country, axis=1)

# filter the dataframe
df_filtered = df[mask]
df_filtered.head()
df_filtered.to_csv("data/im2gps3k/im2gps3k_places365.csv")

In [23]:
from pathlib import Path
import shutil
Path("data/im2gps3k/filtered_im2gps3k").mkdir(parents=True, exist_ok=True)
for image in df_filtered["IMG_ID"]:
    shutil.copyfile(
        f"data/im2gps3k/im2gps3ktest/{image}",
        f"data/im2gps3k/filtered_im2gps3k/{image}"
    )

In [ ]:
import os.path
import os
import optuna
import pandas as pd
import yaml
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
from utils.utils import MP16Dataset
from utils.G3 import G3
from accelerate import Accelerator, DistributedDataParallelKwargs
from zeroshot_prediction import ZeroShotPredictor
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

TUNE_RESULTS_DIR = "results/tune"

import logging
logging.getLogger("lightning").setLevel(logging.ERROR)

In [ ]:
def get_hyperparameter(trial: optuna.trial.Trial, positional_encoding_type, neural_network_type):

    hparams_pe = {}
    if positional_encoding_type == "projectionrff":
        hparams_pe["projection"] = trial.suggest_categorical("projection", ["ecef", "mercator", "eep"])
        hparams_pe["sigma"] = [2**0, 2**4, 2**8]
    elif positional_encoding_type == "projection":
        hparams_pe["projection"] = trial.suggest_categorical("projection", ["ecef", "mercator", "eep"])
    elif positional_encoding_type == "sh":
        hparams_pe["legendre_polys"] = trial.suggest_int("legendre_polys", 10, 30, step=10)
    else:
        raise ValueError(f"Unsupported encoding type: {positional_encoding_type}")

    hparams_nn = {}
    if neural_network_type == "siren":
        hparams_nn["hidden_dim"] = trial.suggest_int("hidden_dim", 256, 1024, step=256)
        hparams_nn["num_layers"] = trial.suggest_int("num_layers", 1, 3)
    elif neural_network_type == "mlp":
        hparams_nn["hidden_dim"] = trial.suggest_int("hidden_dim", 32, 128, step=32)
    elif neural_network_type == "rffmlp":
        hparams_nn["sigma"] = [2**0, 2**4, 2**8]
        hparams_nn["hidden_dim"] = trial.suggest_int("hidden_dim", 32, 128, step=32)
    else:
        raise ValueError(f"Unsupported network type: {neural_network_type}")

    hparams_opt = {}
    hparams_opt["lr"] = trial.suggest_float("lr", 1e-6, 1e-4, log=True)
    hparams_opt["wd"] = trial.suggest_float("wd", 5e-7, 5e-6, log=True)

    hparams = {}
    hparams.update(hparams_pe)
    hparams.update(hparams_nn)
    hparams["optimizer"] = hparams_opt
    
    hparams['harmonics_calculation'] = "analytic"
    
    return hparams

In [ ]:
def train_1epoch(train_loader, val_loader, earlystopper, model, vision_processor, text_processor, optimizer, scheduler, device, accelerator=None):
    # === Training ===
    model.train()
    t = tqdm(train_loader, disable=not accelerator.is_local_main_process)
    for i, (images, texts, longitude, latitude) in enumerate(t):
        texts = text_processor(text=texts, padding='max_length', truncation=True, return_tensors='pt', max_length=77)
        images = images.to(device)
        texts = texts.to(device)
        longitude = longitude.to(device).float()
        latitude = latitude.to(device).float()
        
        optimizer.zero_grad()
        output = model(images, texts, longitude, latitude, return_loss=True)
        loss = output['loss']
        
        # Gradient step
        if accelerator:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()

        # Logging
        if i % 1 == 0:
            t.set_description(f"Train | step {i}, loss {loss.item():.4f}, lr {scheduler.get_last_lr()[0]:.2e}")
    
    scheduler.step()

    # === Validation ===
    model.eval()
    val_losses = []
    with torch.no_grad():
        for images, texts, longitude, latitude in val_loader:
            texts = text_processor(text=texts, padding='max_length', truncation=True, return_tensors='pt', max_length=77)
            images = images.to(device)
            texts = texts.to(device)
            longitude = longitude.to(device).float()
            latitude = latitude.to(device).float()

            output = model(images, texts, longitude, latitude, return_loss=True)
            val_losses.append(output['loss'].item())

    val_loss_avg = sum(val_losses) / len(val_losses)
    print(f"Validation Loss: {val_loss_avg:.4f}")
    return val_loss_avg

In [ ]:
def tune(positional_encoding_type, neural_network_type, dataset_name="mp16"):
    n_trials = 5
    timeout = 90 * 60 # seconds
    epochs = 5

    def objective(trial: optuna.trial.Trial) -> float:
        hparams = get_hyperparameter(trial, positional_encoding_type, neural_network_type)

        ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
        accelerator = Accelerator(kwargs_handlers=[ddp_kwargs])

        device = accelerator.device
        model = G3(
            device, 
            positional_encoding_type,
            neural_network_type,
            hparams=hparams,
        ).to(device)

        # Initialize dataset
        dataset = MP16Dataset(
            vision_processor=model.vision_processor,
            text_processor=model.text_processor,
            root_path='data/mp16/',
            image_data_path='filtered_mp16.tar'
        )

        # Split dataset
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(
            dataset, [train_size, val_size],
            generator=torch.Generator().manual_seed(42)
        )

        # Create dataloaders
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=16, pin_memory=True, prefetch_factor=5)
        val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=16, pin_memory=True, prefetch_factor=5)



        params = []
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name, param.size())
                params.append(param)

        optimizer = torch.optim.AdamW([param for name,param in model.named_parameters() if param.requires_grad], lr=hparams['optimizer']['lr'], weight_decay=hparams['optimizer']['wd'])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.87)

        model, optimizer, train_loader, val_loader, scheduler = accelerator.prepare(
            model, optimizer, train_loader, val_loader, scheduler
        )

        earlystopper = None

        for epoch in range(epochs):
            val_loss = train_1epoch(train_loader, val_loader, earlystopper, model, model.vision_processor, model.text_processor, optimizer, scheduler, device, accelerator)

            trial.report(val_loss, step=epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()

        return val_loss

    pruner = optuna.pruners.MedianPruner()
    study_name = f"{dataset_name}-{positional_encoding_type}-{neural_network_type}"
    os.makedirs(f"{TUNE_RESULTS_DIR}/{dataset_name}/runs/", exist_ok=True)
    storage_name = f"sqlite:///{TUNE_RESULTS_DIR}/{dataset_name}/runs/{study_name}.db"
    study = optuna.create_study(study_name=study_name, direction="minimize", 
                                storage=storage_name, load_if_exists=True, 
                                pruner=pruner)
    study.optimize(objective, n_trials=n_trials, timeout=timeout)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    study.trials_dataframe()

    runsummary = f"{TUNE_RESULTS_DIR}/{dataset_name}/runs/{positional_encoding_type}-{neural_network_type}.csv"
    os.makedirs(os.path.dirname(runsummary), exist_ok=True)

    study.trials_dataframe().to_csv(runsummary)

In [ ]:
def compile_summaries(dataset="mp16"):
    tune_results_dir_this_datset = os.path.join(TUNE_RESULTS_DIR, dataset)
    runsdir = os.path.join(TUNE_RESULTS_DIR, f"{dataset}/runs")

    csvs = [csv for csv in os.listdir(runsdir) if csv.endswith("csv") and csv != "summary.csv"]

    summary = []
    hparams = {}
    for csv in csvs:
        df = pd.read_csv(os.path.join(runsdir, csv))
        best_run = df.sort_values(by="value").iloc[0]
        value = best_run.value
        params = {k.replace("params_", ""): v for k, v in best_run.to_dict().items() if "params" in k}
        pe, nn = csv.replace(".csv", "").split("-")
        hparams[f"{pe}-{nn}"] = params

        sum = {
            "pe":pe,
            "nn":nn,
            "value":value
        }
        sum.update(params)

        summary.append(sum)

    summary = pd.DataFrame(summary).sort_values("value").set_index(["pe","nn"])
    summary.to_csv(os.path.join(tune_results_dir_this_datset, "summary.csv"))

    print("writing " + os.path.join(tune_results_dir_this_datset, "hparams.yaml"))
    with open(os.path.join(tune_results_dir_this_datset, "hparams.yaml"), 'w') as f:
        yaml.dump(hparams, f)

    value_matrix = pd.pivot_table(summary.value.reset_index(), index="pe", columns="nn", values=["value"])["value"]
    print("writing " + os.path.join(tune_results_dir_this_datset, "values.csv"))
    value_matrix.to_csv(os.path.join(tune_results_dir_this_datset, "values.csv"))

    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.imshow(value_matrix)
    ax.set_xticks(range(len(value_matrix.columns)))
    ax.set_xticklabels(value_matrix.columns)
    ax.set_xlabel(value_matrix.columns.name)

    ax.set_yticks(range(len(value_matrix.index)))
    ax.set_yticklabels(value_matrix.index)
    ax.set_ylabel(value_matrix.index.name)

    plt.tight_layout()

    print("writing "+os.path.join(tune_results_dir_this_datset, "values.png"))
    fig.savefig(os.path.join(tune_results_dir_this_datset, "values.png"), transparent=True, bbox_inches="tight", pad_inches=0)

In [ ]:
positional_encoders = ["sh"]
neural_networks = ["siren"]
for pe in positional_encoders:
    for nn in neural_networks:
        tune(pe, nn)

compile_summaries()

[I 2025-05-25 08:27:36,358] A new study created in RDB with name: mp16-sh-siren


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

read text data success
no exist tar index success, need building...


42764it [00:01, 29798.99it/s]


tar index buidling success
data columns:  42764
location from str to float success
logit_scale1 torch.Size([])
logit_scale2 torch.Size([])
logit_scale3 torch.Size([])
location_encoder.neural_network.0.layers.0.weight torch.Size([768, 900])
location_encoder.neural_network.0.layers.0.bias torch.Size([768])
location_encoder.neural_network.0.layers.1.weight torch.Size([768, 768])
location_encoder.neural_network.0.layers.1.bias torch.Size([768])
location_encoder.neural_network.0.last_layer.weight torch.Size([512, 768])
location_encoder.neural_network.0.last_layer.bias torch.Size([512])
vision_projection_else_1.0.weight torch.Size([768, 768])
vision_projection_else_1.0.bias torch.Size([768])
vision_projection_else_1.2.weight torch.Size([768, 768])
vision_projection_else_1.2.bias torch.Size([768])
text_projection_else.0.weight torch.Size([768, 768])
text_projection_else.0.bias torch.Size([768])
text_projection_else.2.weight torch.Size([768, 768])
text_projection_else.2.bias torch.Size([768])


step 7, loss 11.326229095458984, lr 3e-05:   5%|▍         | 8/168 [02:48<27:25, 10.28s/it]   